#### Create Data Generator

In [14]:
class DataGen(tf.keras.utils.Sequence):
    def __init(self, ids, path, batch_size=8, image_size=64):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
    
    def __load__(self, id_name):
        image_path = os.path.join(self.path, id_name, "images", id_name) + ".tif"
        mask_path = os.path.join(self.path, id_name, "masks/")
        all_masks = os.listdir(mask_path)
        
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        
        masks = np.zeros((self.image_size, self.image_size, 1))
        
        for name in all_masks:
            _mask_path = mask_path + name
            _mask_image = cv2.imread(_mask_path, -1) #greyscale
            _mask_image = cv2.resize(image, (self.image_size, self.image_size)) # 64x64
            _mask_image = np.expand_dims(_mask_image, axis=1)
            mask = np.maximum(mask, _mask_image)
        
        # normalize image
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask = []
        
        for id_name in file_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
        
        image = np.array(image)
        mask = np.array(mask)
        
        return image, mask

    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

#### Set Hyperparameters

In [15]:
image_size = 64
train_path = '/home/jovyan/ghw2019_planetpieces/contributors/claire/unet-model-claire/data/train'
epochs = 1
batch_size = 8

train_ids = next(os.walk(train_path))[1]

val_data_size = 5

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [ ]:
gen = DataGen(train_ids, train_path,
              batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

In [2]:
class UNet():
    def __init__(self):
        print ('build UNet ...')

    def get_crop_shape(self, target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

    def create_model(self, img_shape, num_class):

        concat_axis = 3
        inputs = tf.keras.layers.Input(shape = img_shape)

        conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
        conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

        up_conv5 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv5)
        ch, cw = self.get_crop_shape(conv4, up_conv5)
        crop_conv4 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(conv4)
        up6 = tf.keras.layers.concatenate([up_conv5, crop_conv4], axis=concat_axis)
        conv6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conv6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

        up_conv6 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv6)
        ch, cw = self.get_crop_shape(conv3, up_conv6)
        crop_conv3 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(conv3)
        up7 = tf.keras.layers.concatenate([up_conv6, crop_conv3], axis=concat_axis) 
        conv7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conv7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

        up_conv7 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv7)
        ch, cw = self.get_crop_shape(conv2, up_conv7)
        crop_conv2 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(conv2)
        up8 = tf.keras.layers.concatenate([up_conv7, crop_conv2], axis=concat_axis)
        conv8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conv8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

        up_conv8 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv8)
        ch, cw = self.get_crop_shape(conv1, up_conv8)
        crop_conv1 = tf.keras.layers.Cropping2D(cropping=(ch,cw))(conv1)
        up9 = tf.keras.layers.concatenate([up_conv8, crop_conv1], axis=concat_axis)
        conv9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conv9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

        ch, cw = self.get_crop_shape(inputs, conv9)
        conv9 = tf.keras.layers.ZeroPadding2D(padding=((ch[0], ch[1]), (cw[0], cw[1])))(conv9)
        conv10 = tf.keras.layers.Conv2D(num_class, (1, 1))(conv9)

        model = tf.keras.models.Model(inputs=inputs, outputs=conv10)

        return model